# Middleware Redaction and Retry

    - Goal: Redact PII before logging
- Goal: Retry transient errors with exponential backoff

    **Pass criteria:** `events.json` records retry history with redactions tracked.

    _Mock mode uses fixtures and writes artifacts under `_runs/`. Flip `USE_REAL_CLIENT=True` when ready for live calls._

In [ ]:

USE_REAL_CLIENT = False  # flip when ready
import os, json, time, pathlib, math, random

RUN = pathlib.Path('_runs/05_middleware')
RUN.mkdir(parents=True, exist_ok=True)
FIXTURE_ROOT = pathlib.Path(os.environ.get('MOCK_FIXTURE_ROOT', '_fixtures'))

def write_meta(**kwargs):
    payload = {"timestamp": time.time(), **kwargs}
    (RUN / 'meta.json').write_text(json.dumps(payload, indent=2))
    return payload

USE_REAL_CLIENT = bool(str(os.environ.get('USE_REAL_CLIENT', USE_REAL_CLIENT)).lower() in {'1', 'true', 'yes'})


In [ ]:
from tools.redact_middleware import apply_redaction
from tools.backoff import retry
message = [{"role": "user", "content": "Please email me at learner@example.com with the plan."}]
cleaned = apply_redaction(message)
(RUN / "redacted.json").write_text(json.dumps(cleaned, indent=2))
counter = {"attempts": 0}
def flaky_operation():
    counter["attempts"] += 1
    if counter["attempts"] < 2:
        raise RuntimeError("mock transient error")
    return "processed"
result, history = retry(flaky_operation, attempts=3, sleeper=lambda _: None)
events = [{"attempt": h.index, "delay": round(h.delay, 3), "success": h.success, "error": h.error} for h in history]
events_path = RUN / "events.json"
events_path.write_text(json.dumps({"events": events, "result": result}, indent=2))
events

In [ ]:
meta = write_meta(ok=True, lab="05_middleware", attempts=len(events), redactions=cleaned[0]['redactions'])
meta